# Package Install
The package installation section establishes the technological foundation through essential libraries: `numpy` enables efficient numerical array operations and mathematical computations fundamental to scientific computing; `pandas` facilitates structured data manipulation and tabular analysis with powerful DataFrame operations; `seaborn` and `matplotlib` provide statistical visualisations and graphical representations essential for exploratory data analysis and result communication; `scikit-learn` supplies machine learning algorithms, preprocessing utilities, and comprehensive model evaluation metrics; and `pyspark` enables distributed computing capabilities for processing large-scale datasets exceeding single-machine memory constraints.

Uncomment the line that corresponds to your Kernel of choice

In [ ]:
# %conda install numpy; pandas; seaborn; scikit-learn; matplotlib; pyspark; imblearn
# %pip install numpy; pandas; seaborn; scikit-learn; matplotlib; pyspark; imblearn

# Import Libraries
The import statements are strategically organised by functional category to establish a comprehensive analytical toolkit. Core data processing leverages `pandas` for DataFrames and `numpy` for numerical computations. Visualisation frameworks encompass `matplotlib.pyplot` for customised plotting and `seaborn` for enhanced statistical graphics. Scikit-learn imports include `RobustScaler` for outlier-resilient feature standardisation, `train_test_split` for stratified dataset partitioning, `GridSearchCV` and `StratifiedKFold` for systematic hyperparameter optimisation and cross-validation, and comprehensive metrics including `confusion_matrix`, `classification_report`, individual scoring functions (`precision_score`, `recall_score`, `f1_score`, `accuracy_score`), and ROC analysis tools (`roc_curve`, `roc_auc_score`) for threshold-independent performance assessment.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, roc_curve, 
                            precision_recall_curve, confusion_matrix, 
                            classification_report, average_precision_score)
from sklearn.utils.validation import check_X_y, check_array
from imblearn.over_sampling import SMOTE
from itertools import product
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

import warnings

warnings.filterwarnings('ignore')

random.seed(42)
np.random.seed(42)

# Data Loading

Dataset: GoodReads 100k books (https://www.kaggle.com/datasets/mdhamani/goodreads-books-100k/data)

The dataset loading process begins by defining target columns representing key book characteristics theoretically relevant for predicting bestseller status: `rating` (numerical quality score), `reviews` (engagement frequency), `pages` (book length), `totalratings` (popularity measure), and optional categorical fields (`genre`, `author`, `title`). The implementation starts by using Pyspark but it turns out that there is data corruption caused by Pyspark reading the data. Thus pandas is used with `pd.read_csv()` configured to selectively load columns via `usecols` parameter, optimising memory usage by excluding irrelevant fields during the read operation. This approach ensures data integrity and preserves correct field alignment, addressing limitations encountered with distributed processing frameworks.

In [ ]:
# Set paths and constants
DATA_PATH = "GoodReads_100k_books.csv"
RANDOM_STATE = 42
try:
    # Initialize Spark Session
    spark = SparkSession.builder.appName("DataLoadTest").getOrCreate()
    
    # Load the dataset using PySpark with default schema inference
    df_spark_test = spark.read.csv(
        DATA_PATH, 
        header=True, 
        inferSchema=True
    )
    
    print("\nPySpark Inferred Schema (Note potential incorrect types or nulls):")
    df_spark_test.printSchema()
    
    print("\nFirst 5 rows showing potential type conversion problems:")
    df_spark_test.select('rating', 'reviews', 'pages').limit(5).show()
    
    # Clean up the Spark session
    spark.stop()

except Exception as e:
    print(f"Could not initialize or load with PySpark (PySpark may not be installed or configured): {e}")

## Important Note:

As shown above and overall tested, using the dataset originally loaded using pyspark and then converting it to Pandas, causes the data to be misread and thus skews the results of the model. As shown above, certain values are in the wrong fields as well as certain values are being read as `NULL` when there is a value in that field within the dataset. Along with that, upon analysing the data and processing it, the numerical values were not converted to numerical fields and attempting to do so later in the code, does not work the way it needs to. Thus causing me to have to load the dataset using Pandas, while still keeping the pyspark code in as it is required for this POE.

In [ ]:
# Load the dataset using pandas for robustness and explicit control
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Data loaded successfully with Pandas. Shape: {df.shape}")
except FileNotFoundError:
    print("Error: CSV file not found. Check DATA_PATH.")
    df = pd.DataFrame()

# Display initial information
if not df.empty:
    print("\nInitial Pandas Data Info:")
    df.info()
    print("\nFirst 5 rows:")
    print(df.head())

In [ ]:
df.info()
print("\n")
df.describe()

# EDA
Exploratory Data Analysis encompasses comprehensive investigation of dataset structure, characteristics, quality issues, and relationships between variables. This section systematically characterises the dataset through multiple complementary analytical perspectives including distributional analysis, relationship identification, and anomaly detection to inform subsequent preprocessing and feature engineering decisions.

## Define Bestseller

Best seller (in the context of a book) is a book that is among those having the largest sales during a given period.

Deriving from that definition, a book with a high rating and review count would indicate popularity, thus indicating a best selling book. Using that thinking, a book is identified as a best seller based if it falls in the top 25% of both number of reviews and number of ratings, indicated by the columns `reviews` and `totalratings`.

Originally it was going to be based on the `rating` of the book, then changed to the `rating` and `totalratings` but after studying the values in the dataset, it was shown that a best seller does not mean a 5 star book. 
This is proven by the book 'Fifty Shades of Grey' by E.L. James (https://www.goodreads.com/book/show/10818853-fifty-shades-of-grey)
The book is a best seller has 2 million plus ratings, 86 thousand reviews but is only rated 3.67 (as of 30 October 2025). 

In [ ]:
# Definition of best seller in relation to the dataset, which does not contain a sales field
df['bestseller'] = ((df['reviews'] >= df['reviews'].quantile(0.75)) & 
                    (df['totalratings'] >= df['totalratings'].quantile(0.75))).astype(int)

## Bestseller Variable Analysis

In [ ]:
bs_counts = df['bestseller'].value_counts()

fig, ax = plt.subplots(1, 2, figsize=(14, 5))
# Count plot
bs_counts.plot(kind='bar', ax=ax[0], color=['#FF6B6B', '#4ECDC4'])
ax[0].set_title('Bestseller Variable Distribution', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Class (0: Non-Bestseller, 1: Bestseller)', fontsize=12)
ax[0].set_ylabel('Count', fontsize=12)
ax[0].tick_params(rotation=0)

# Pie chart
ax[1].pie(bs_counts, labels=['Non-Bestseller', 'Bestseller'], 
          autopct='%1.1f%%', colors=['#FF6B6B', '#4ECDC4'], startangle=90)
ax[1].set_title('Bestseller Variable Proportion', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Univariate Analysis
Univariate analysis examines individual features independently through histograms with kernel density estimation to visualise distributional properties including modality, skewness, and outlier presence. This component-by-component investigation identifies heavily skewed distributions requiring transformation, bimodal or multimodal distributions suggesting potential data quality issues or heterogeneous populations, and extreme outliers necessitating treatment to prevent model distortion.

In [ ]:
# Identify numerical columns
numerical_cols = ['rating', 'reviews', 'pages', 'totalratings']
# Filter to only existing columns
numerical_cols = [col for col in numerical_cols if col in df.columns]

# Convert to numeric if needed
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"\nNumerical Features Summary:")
print(df[numerical_cols].describe())

In [ ]:
# Histograms and KDE plots for numerical features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    if idx < len(axes):
        # Drop NaNs and zero values
        col_data = df[col].dropna()
        filtered_data = col_data[col_data != 0]

        # Remove outliers (top 1%)
        upper_limit = filtered_data.quantile(0.99)
        filtered_data = filtered_data[filtered_data <= upper_limit]

        # Plot histogram
        ax = axes[idx]
        filtered_data.hist(bins=50, ax=ax, color='skyblue', edgecolor='black', alpha=0.7)

        # KDE plot
        ax2 = ax.twinx()
        filtered_data.plot(kind='kde', ax=ax2, color='red', linewidth=2)

        # Labels and titles
        ax.set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
        ax.set_xlabel(col, fontsize=10)
        ax.set_ylabel('Frequency', fontsize=10)
        ax2.set_ylabel('Density', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Categorical features analysis
if 'genre' in df.columns:
    print(f"\nGenre Analysis:")
    # Genres are often comma-separated, so let's count occurrences
    all_genres = []
    for genres in df['genre'].dropna():
        all_genres.extend([g.strip() for g in str(genres).split(',')])

    genre_counts = pd.Series(all_genres).value_counts().head(10)
    print("Top 10 Most Common Genres:")
    print(genre_counts)

    # Plot top genres
    plt.figure(figsize=(12, 6))
    genre_counts.plot(kind='bar', color='coral')
    plt.title('Top 10 Most Common Genres', fontsize=14, fontweight='bold')
    plt.xlabel('Genre', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Book format analysis
if 'bookformat' in df.columns:
    print(f"\nBook Format Distribution:")
    format_counts = df['bookformat'].value_counts()
    print(format_counts)

## Multivariate Analysis
Multivariate analysis investigates relationships and interactions between multiple variables through correlation matrices, heatmap visualisations, and boxplots stratified by target class. These analyses reveal feature dependencies that may indicate redundancy, identify variables exhibiting strong differential distributions between classes (suggesting high predictive power), detect potential confounding relationships, and reveal interactions not apparent in univariate examination.

In [ ]:
# Correlation matrix
correlation_matrix = df[numerical_cols].corr()
print("\nCorrelation Matrix:")
print(correlation_matrix)

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.ticker as ticker
# Box plots: Numerical features by Bestseller class
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    if idx < len(axes):
        df.boxplot(column=col, by='bestseller', ax=axes[idx])
        axes[idx].set_title(f'{col} by Bestseller Class')
        plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}"))
        axes[idx].set_xlabel('Bestseller')
        axes[idx].set_ylabel(col)

plt.suptitle('Box Plots of Numerical Features by Bestseller Class')
plt.tight_layout()
plt.show()


In [ ]:
# Violin plots for better distribution visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    sns.violinplot(data=df, x='bestseller', y=col, ax=axes[idx], hue='bestseller', palette='Set2', legend=False)
    axes[idx].set_xticks([0, 1])
    axes[idx].set_xticklabels(['Non-Bestseller', 'Bestseller'])
    axes[idx].set_title(f'Distribution of {col} by Bestseller')
plt.tight_layout()
plt.show()

# Data Cleaning and Feature Analysis
Comprehensive preprocessing pipeline preparation systematically addresses data quality issues through duplicate detection and removal, missing value imputation using appropriate strategies matched to feature types and statistical properties, outlier treatment using robust statistical methods like interquartile range, and feature engineering creating derived features capturing complex relationships and non-linearities.

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Calculate number of duplicate rows
num_duplicates = df_processed.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Drop duplicate rows
df_processed = df_processed.drop_duplicates()

# Confirm removal
print(f"Shape after dropping duplicates: {df_processed.shape}")

## Handle Missing values
Missing value treatment employs strategy differentiation based on feature characteristics and missingness mechanisms. Numerical features undergo median imputation preserving distributional properties resistant to outlier influence. Categorical features receive mode imputation or placeholder values ('Unknown') maintaining category structure. The approach prioritises retaining information whilst maintaining data integrity and minimising model bias introduced by inappropriate imputation.

In [ ]:
# Rating: fill with median
if df_processed['rating'].isnull().sum() > 0:
    df_processed['rating'].fillna(df_processed['rating'].median(), inplace=True)

# Reviews: fill with 0 (no reviews)
if df_processed['reviews'].isnull().sum() > 0:
    df_processed['reviews'].fillna(0, inplace=True)

# Pages: fill with median
if df_processed['pages'].isnull().sum() > 0:
    df_processed['pages'].fillna(df_processed['pages'].median(), inplace=True)

# Total ratings: fill with 0
if df_processed['totalratings'].isnull().sum() > 0:
    df_processed['totalratings'].fillna(0, inplace=True)

# Categorical: fill with 'Unknown'
for col in ['author', 'genre', 'title']:
    if col in df_processed.columns and df_processed[col].isnull().sum() > 0:
        df_processed[col] = df_processed[col].fillna('Unknown')
        
df_processed = df_processed.drop(columns=['isbn', 'isbn13', 'link', 'img', 'desc'])

## Outlier Treatment
Outlier detection and treatment employs the Interquartile Range (IQR) method calculating bounds from quartile measurements: lower_bound = Q1 - 1.5×IQR and upper_bound = Q3 + 1.5×IQR. Observations beyond these bounds are identified as outliers. Treatment approaches include removing extreme values, windsorising (capping at bounds), or log-transformation reducing outlier influence whilst preserving information for highly right-skewed distributions common in engagement metrics.

In [ ]:
def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect and report outliers
for col in numerical_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_pct = (len(outliers) / len(df)) * 100

## Feature Engineering
Comprehensive feature engineering spans multiple categories: logarithmic transformations (`np.log1p()`) address heavily skewed distributions preserving zero values; ratio features capture relationships between variables; binning converts continuous variables into discrete categories; binary indicators create flags for specific conditions; text features extract information from string fields; and interaction features capture multiplicative relationships not evident from individual components. These engineered features often provide superior predictive power through exposing non-linear relationships and interaction effects.

In [ ]:
# Create log-transformed features for skewed distributions
if 'reviews' in df.columns:
    df['log_reviews'] = np.log1p(df['reviews'])  # log(1 + x) to handle zeros

if 'totalratings' in df.columns:
    df['log_totalratings'] = np.log1p(df['totalratings'])

# Create title length feature
if 'title' in df.columns:
    df['title_length'] = df['title'].astype(str).apply(len)

# Create interaction term
if 'rating' in df.columns and 'totalratings' in df.columns:
    df['rating_x_totalratings'] = df['rating'] * df['totalratings']

# Create review engagement ratio
if 'reviews' in df.columns and 'totalratings' in df.columns:
    df['review_engagement_ratio'] = df['reviews'] / (df['totalratings'] + 1)


In [ ]:
# Display new features
print("\n" + "="*40)
print("NEW FEATURES CREATED:")
print("="*40)
new_features = ['log_reviews', 'log_totalratings', 'title_length', 
                'rating_x_totalratings', 'review_engagement_ratio']
new_features = [f for f in new_features if f in df.columns]
print(f"Created {len(new_features)} new features:")
for feature in new_features:
    print(f"  - {feature}")

if new_features:
    print("\nSample of new features:")
    print(df[new_features].head())

## Finalize Bestseller Variable
Target variable preparation ensures consistent data type and removes any remaining inconsistencies. The target vector is extracted as a NumPy array suitable for model consumption.

In [ ]:
# Define y as the Bestseller 
y = df['bestseller'].values

# Model Definition
Custom logistic regression implementation from first principles, incorporating gradient descent optimisation with L2 regularisation for overfitting prevention. The implementation includes probability prediction through sigmoid transformation, class label prediction with configurable threshold, loss history tracking for convergence monitoring, and coefficient extraction for feature importance interpretation.

## Logistical Regression Model: Model Training and Prediction
Custom logistic regression classifier implementation using gradient descent for parameter optimisation. The sigmoid activation function transforms linear combinations into probabilities bounded within [0,1]. Binary cross-entropy loss with L2 regularisation penalty guides parameter updates. The implementation enables probability estimation and classification prediction with flexible threshold specification.

In [ ]:
class LogisticRegressionScratch:
    """
    Custom Logistic Regression implementation with L2 regularization.

    Parameters:
    -----------
    lr : float
        Learning rate for gradient descent
    epochs : int
        Number of iterations for training
    lambda_reg : float
        Regularization parameter (L2)
    verbose : bool
        Whether to print training progress
    """

    def __init__(self, lr=0.01, epochs=1000, lambda_reg=0.1, verbose=True):
        self.lr = lr
        self.epochs = epochs
        self.lambda_reg = lambda_reg
        self.verbose = verbose
        self.weights = None
        self.bias = None
        self.loss_history = []
        self.accuracy_history = []
        self.val_accuracy_history = []


    def sigmoid(self, z):
        """
        Sigmoid activation function: σ(z) = 1 / (1 + e^(-z))
        """
        # Clip to prevent overflow
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def compute_loss(self, y_true, y_pred, weights):
        """
        Compute binary cross-entropy loss with L2 regularization:
        L = -1/m * Σ(y*log(ŷ) + (1-y)*log(1-ŷ)) + λ/(2m) * ||w||²
        """
        m = len(y_true)
        # Add small epsilon to prevent log(0)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

        # Binary cross-entropy
        bce_loss = -np.mean(y_true * np.log(y_pred) + 
                           (1 - y_true) * np.log(1 - y_pred))

        # L2 regularization term (excluding bias)
        l2_penalty = (self.lambda_reg / (2 * m)) * np.sum(weights ** 2)

        total_loss = bce_loss + l2_penalty
        return total_loss

    def compute_gradients(self, X, y_true, y_pred):
        """
        Compute gradients for weights and bias with L2 regularization:
        ∂L/∂w = 1/m * X^T(ŷ - y) + λ/m * w
        ∂L/∂b = 1/m * Σ(ŷ - y)
        """
        m = X.shape[0]
        err = y_pred - y_true

        # Gradient for weights (with L2 regularization)
        dw = (1/m) * np.dot(X.T, err) + (self.lambda_reg/m) * self.weights

        # Gradient for bias (no regularization on bias)
        db = (1/m) * np.sum(err)

        return dw, db
    
    def fit(self, X, y, X_val=None, y_val=None):
        """
        Train the logistic regression model using gradient descent.
        Tracks both training and (optionally) validation loss.

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Training data
        y : array-like, shape (n_samples,)
            Training labels (0 or 1)
        X_val : array-like, optional
            Validation data
        y_val : array-like, optional
            Validation labels
        """
        X, y = check_X_y(X, y)
        n_samples, n_features = X.shape

        self.weights = np.zeros(n_features)
        self.bias = 0

        self.loss_history = []
        self.val_loss_history = []
        self.accuracy_history = []
        self.val_accuracy_history = []

        for epoch in range(self.epochs):
            #--- Training prediction & loss ---
            lin_out = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(lin_out)
            loss = self.compute_loss(y, y_pred, self.weights)
            self.loss_history.append(loss)

            # Track training accuracy
            train_acc = np.mean((y_pred >= 0.5).astype(int) == y)
            self.accuracy_history.append(train_acc)
            
            #--- Validation prediction & loss (if validation set provided) ---
            if X_val is not None and y_val is not None:
                linear_val = np.dot(X_val, self.weights) + self.bias
                y_val_pred = self.sigmoid(linear_val)
                val_loss = self.compute_loss(y_val, y_val_pred, self.weights)
                self.val_loss_history.append(val_loss)
                
                # Track validation accuracy
                val_acc = np.mean((y_val_pred >= 0.5).astype(int) == y_val)
                self.val_accuracy_history.append(val_acc)

            #--- Gradients and update ---
            dw, db = self.compute_gradients(X, y, y_pred)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

            if self.verbose and (epoch % 100 == 0 or epoch == self.epochs - 1):
                acc = np.mean((y_pred >= 0.5).astype(int) == y)
                logstr = f"Epoch {epoch:4d}/{self.epochs} | Loss: {loss:.4f} | Accuracy: {acc:.4f}"
                # Print validation loss if provided
                if X_val is not None and y_val is not None:
                    logstr += f" | Val Loss: {val_loss:.4f}"
                print(logstr)
        return self

    def predict_proba(self, X):
        """
        Predict class probabilities for X.

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Test data

        Returns:
        --------
        prob : array, shape (n_samples,)
            Probability of positive class
        """
        X = check_array(X)
        linear_output = np.dot(X, self.weights) + self.bias
        prob = self.sigmoid(linear_output)
        return prob

    def predict(self, X, threshold):
        """
        Predict class labels for X.

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Test data
        threshold : float
            Classification threshold

        Returns:
        --------
        pred : array, shape (n_samples,)
            Predicted class labels (0 or 1)
        """
        prob = self.predict_proba(X)
        pred = (prob >= threshold).astype(int)
        return pred

    def get_coefficients(self):
        """
        Get model coefficients (weights and bias).

        Returns:
        --------
        dict : Dictionary containing weights and bias
        """
        return {
            'weights': self.weights,
            'bias': self.bias
        }

# Data Transformation Pipeline
Feature transformations prepare data for model input through categorical encoding converting categories into numerical representations and scaling standardising feature ranges to similar magnitudes, improving convergence and preventing large-magnitude features from dominating model decisions.

## Train Test Split
Dataset partitioning divides data into training (60%), validation (20%) and testing (20%) subsets using stratified random splitting ensuring both subsets maintain identical class distribution proportions. Stratification prevents class imbalance exacerbation in small test sets. The `random_state=42` parameter ensures reproducibility enabling consistent results across multiple executions.

In [ ]:
# Select features for modeling

# Numerical features to include if present in dataset
numerical_features = [
    'rating', 'reviews', 'pages', 'totalratings',
    'log_reviews', 'log_totalratings', 'title_length',
    'rating_x_totalratings', 'review_engagement_ratio',
    'is_highly_rated'
]
numerical_features = [f for f in numerical_features if f in df.columns]

# Categorical features to encode if present in dataset
categorical_features = ['genre']
categorical_features = [f for f in categorical_features if f in df.columns]

# Build the feature set
all_features = numerical_features + categorical_features
X = df[all_features]

# Replace inf/-inf with large numbers for safety, and fill NaNs
X = X.replace([np.inf, -np.inf], [1e10, -1e10]).fillna(0)

In [ ]:
# Split data into train+val and test first
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Then split train+val into train and val
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape:   {X_val.shape}")
print(f"X_test shape:  {X_test.shape}")

## One Hot Encoding
Categorical feature encoding transforms categorical variables into numerical representations through one-hot encoding creating binary indicator variables for each category with `drop_first=True` avoiding the dummy variable trap causing multicollinearity. This process standardises feature space for model consumption.

In [ ]:
categorical_features = [col for col in categorical_features if 
                        col in X_train.columns and col in X_val.columns and col in X_test.columns]

if categorical_features:
    # Clean up text in categorical columns
    for col in categorical_features:
        X_train[col] = X_train[col].astype(str).str.split(',').str[0].str.strip()
        X_val[col] = X_val[col].astype(str).str.split(',').str[0].str.strip()
        X_test[col] = X_test[col].astype(str).str.split(',').str[0].str.strip()

    # Filter only low-cardinality features
    low_cardinality = [col for col in categorical_features if X_train[col].nunique() <= 50]

    if low_cardinality:
        # Encode and align
        X_train_enc = pd.get_dummies(X_train[low_cardinality], drop_first=True)
        X_val_enc = pd.get_dummies(X_val[low_cardinality], drop_first=True)
        X_test_enc = pd.get_dummies(X_test[low_cardinality], drop_first=True)

        # Align columns across all three sets
        X_train_enc, X_val_enc = X_train_enc.align(X_val_enc, join='left', axis=1, fill_value=0)
        X_train_enc, X_test_enc = X_train_enc.align(X_test_enc, join='left', axis=1, fill_value=0)
        X_val_enc, X_test_enc = X_val_enc.align(X_test_enc, join='left', axis=1, fill_value=0)

        # Drop original categorical columns
        X_train = X_train.drop(columns=low_cardinality)
        X_val = X_val.drop(columns=low_cardinality)
        X_test = X_test.drop(columns=low_cardinality)

        # Concatenate encoded columns back
        X_train = pd.concat([X_train.reset_index(drop=True), X_train_enc.reset_index(drop=True)], axis=1)
        X_val = pd.concat([X_val.reset_index(drop=True), X_val_enc.reset_index(drop=True)], axis=1)
        X_test = pd.concat([X_test.reset_index(drop=True), X_test_enc.reset_index(drop=True)], axis=1)
        
    X_train = X_train.select_dtypes(exclude=['object'])
    X_val = X_val.select_dtypes(exclude=['object'])
    X_test = X_test.select_dtypes(exclude=['object'])
else:
    print("No categorical features to encode.")

# After encoding, assert only numerics remain
assert X_train.select_dtypes(include='object').shape[1] == 0, "Non-numeric columns remain after encoding!"
assert X_val.select_dtypes(include='object').shape[1] == 0, "Non-numeric columns remain after encoding!"
assert X_test.select_dtypes(include='object').shape[1] == 0, "Non-numeric columns remain after encoding!"


In [ ]:
X_train_values = X_train.values
X_val_values = X_val.values
X_test_values = X_test.values

feature_names = X_train.columns.tolist()

## Feature Scaling
Feature scaling using RobustScaler standardises feature ranges through robust statistical transformations (median centering and Interquartile Range normalisation) less sensitive to outliers than standard scaling. The scaler is fit exclusively on training data preventing data leakage, then applied identically to validation and test sets maintaining distributional consistency.

In [ ]:
# Initialize RobustScaler (less sensitive to outliers than StandardScaler)
scaler = RobustScaler()

print("Fitting scaler on training data...")
# Fit scaler on training data ONLY
scaler.fit(X_train_values)

# Transform train, validation, and test data
X_train_scaled = scaler.transform(X_train_values)
X_val_scaled = scaler.transform(X_val_values)
X_test_scaled = scaler.transform(X_test_values)

print(f"\nScaled data shapes:")
print(f"X_train_scaled: {X_train_scaled.shape}")
print(f"X_val_scaled:   {X_val_scaled.shape}")
print(f"X_test_scaled:  {X_test_scaled.shape}")

# Display scaling statistics
print(f"\nScaling statistics (from training data):")
print(f"Median (center) - first 5 features: {scaler.center_[:5]}")
print(f"IQR (scale) - first 5 features: {scaler.scale_[:5]}")

# Verify scaling worked
print(f"\nTraining data after scaling:")
print(f"Mean: {X_train_scaled.mean(axis=0)[:5]}")  # Should be close to 0
print(f"Std: {X_train_scaled.std(axis=0)[:5]}")    # Should be close to 1


# Initial Model Training and Evaluation
Training and evaluation phase establishing baseline performance metrics before hyperparameter optimisation. This phase trains a model with preliminary hyperparameters and comprehensively evaluates performance across training and test datasets using multiple metrics.

## Train Model
Model instantiation and training with preliminary hyperparameters establishes baseline learning behaviour. The verbose output enables real-time loss monitoring facilitating convergence assessment and identification of training issues.

In [ ]:
# Initialize model with preliminary hyperparameters
initial_model = LogisticRegressionScratch(
    lr=0.01,
    epochs=1000,
    lambda_reg=0.1,
    verbose=True
)

print("\nTraining model with preliminary hyperparameters:")
print(f"  Learning Rate: {initial_model.lr}")
print(f"  Epochs: {initial_model.epochs}")
print(f"  Regularization (λ): {initial_model.lambda_reg}")
print("\n" + "-"*50)

# Train the model
initial_model.fit(X_train_scaled, y_train, X_val=X_val_scaled, y_val=y_val)

## Make Predictions
Prediction generation produces both class labels and probability estimates on training and test datasets enabling comprehensive performance evaluation across multiple perspectives and threshold operating points.

In [ ]:
# Get predictions on both training and validation sets
y_train_pred_initial = initial_model.predict(X_train_scaled, threshold=0.5)
y_val_pred_initial = initial_model.predict(X_val_scaled, threshold=0.5)
y_pred_initial = initial_model.predict(X_test_scaled, threshold=0.5)

# Get probabilities for ROC-AUC calculation
y_train_proba_initial = initial_model.predict_proba(X_train_scaled)
y_val_proba_initial = initial_model.predict_proba(X_val_scaled)
y_proba_initial = initial_model.predict_proba(X_test_scaled)


## Initial Evaluation
Initial performance assessment computes classification metrics including accuracy, precision, recall, F1-score, and ROC AUC across training and test sets. These metrics establish baseline performance for comparison against optimised models.

### Training vs Validation
Comparative assessment of training versus validation performance identifies potential overfitting or underfitting. Training performance substantially exceeding test performance indicates overfitting requiring regularisation strengthening or model simplification.

In [ ]:

# Calculate metrics for training set
train_accuracy_initial = accuracy_score(y_train, y_train_pred_initial)
train_precision_initial = precision_score(y_train, y_train_pred_initial, zero_division=0)
train_recall_initial = recall_score(y_train, y_train_pred_initial, zero_division=0)
train_f1_initial = f1_score(y_train, y_train_pred_initial, zero_division=0)
train_roc_auc_initial = roc_auc_score(y_train, y_train_proba_initial)

# Calculate metrics for validation set
val_accuracy_initial = accuracy_score(y_val, y_val_pred_initial)
val_precision_initial = precision_score(y_val, y_val_pred_initial, zero_division=0)
val_recall_initial = recall_score(y_val, y_val_pred_initial, zero_division=0)
val_f1_initial = f1_score(y_val, y_val_pred_initial, zero_division=0)
val_roc_auc_initial = roc_auc_score(y_val, y_val_proba_initial)

# Print individual metrics for clarity
print("\nINDIVIDUAL TRAINING SET METRICS:")
print(f"  Accuracy:  {train_accuracy_initial:.4f}")
print(f"  Precision: {train_precision_initial:.4f}")
print(f"  Recall:    {train_recall_initial:.4f}")
print(f"  F1-Score:  {train_f1_initial:.4f}")
print(f"  ROC AUC:   {train_roc_auc_initial:.4f}")

print("\nINDIVIDUAL VALIDATION SET METRICS:")
print(f"  Accuracy:  {val_accuracy_initial:.4f}")
print(f"  Precision: {val_precision_initial:.4f}")
print(f"  Recall:    {val_recall_initial:.4f}")
print(f"  F1-Score:  {val_f1_initial:.4f}")
print(f"  ROC AUC:   {val_roc_auc_initial:.4f}")


In [ ]:
epochs = range(1, len(initial_model.loss_history) + 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# LOSS CURVE
axes[0].plot(epochs, initial_model.loss_history, label='Training Loss', color='#2E86AB')
if hasattr(initial_model, 'val_loss_history') and len(initial_model.val_loss_history) > 0:
    axes[0].plot(epochs, initial_model.val_loss_history, label='Validation Loss', color='#A23B72')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training vs Validation Loss (Curve)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# ACCURACY CURVE
if hasattr(initial_model, 'accuracy_history') and len(initial_model.accuracy_history) > 0:
    axes[1].plot(epochs, initial_model.accuracy_history, label='Training Accuracy', color='#2E86AB')
if hasattr(initial_model, 'val_accuracy_history') and len(initial_model.val_accuracy_history) > 0:
    axes[1].plot(epochs, initial_model.val_accuracy_history, label='Validation Accuracy', color='#A23B72')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training vs Validation Accuracy (Curve)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


### Initial Model 
Test set evaluation provides unbiased performance assessment on previously unseen data representing true generalisation capability.

In [ ]:
accuracy_initial = accuracy_score(y_test, y_pred_initial)
precision_initial = precision_score(y_test, y_pred_initial, zero_division=0)
recall_initial = recall_score(y_test, y_pred_initial, zero_division=0)
f1_initial = f1_score(y_test, y_pred_initial, zero_division=0)
roc_auc_initial = roc_auc_score(y_test, y_proba_initial)

print("INITIAL MODEL PERFORMANCE")
print("="*50)
print(f"  Accuracy:  {accuracy_initial:.4f}")
print(f"  Precision: {precision_initial:.4f}")
print(f"  Recall:    {recall_initial:.4f}")
print(f"  F1-Score:  {f1_initial:.4f}")
print(f"  ROC AUC:   {roc_auc_initial:.4f}")

# Confusion Matrix
cm_initial = confusion_matrix(y_test, y_pred_initial)
print("\nConfusion Matrix:")
print(cm_initial)
print("\nConfusion Matrix Breakdown:")
tn, fp, fn, tp = cm_initial.ravel()
print(f"  True Negatives (TN):  {tn}")
print(f"  False Positives (FP): {fp}")
print(f"  False Negatives (FN): {fn}")
print(f"  True Positives (TP):  {tp}")

## The graphs below are based on the initial model's results from the test set

## Confusion Matrix Visualisation
Confusion matrix presentation through annotated heatmap visualisation clearly shows true positives, true negatives, false positives, and false negatives enabling intuitive understanding of classification performance and error patterns across classes.

In [ ]:
cm_initial = confusion_matrix(y_test, y_pred_initial)

# Visualize confusion matrix,
plt.figure(figsize=(8, 6))
sns.heatmap(cm_initial, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Non-Bestseller', 'Bestseller'],
            yticklabels=['Non-Bestseller', 'Bestseller'])
plt.title('Initial Model - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

print("\nConfusion Matrix Values:")
print(f"Initial Model Set TP: {cm_initial[1,1]}, TN: {cm_initial[0,0]}, FP: {cm_initial[0,1]}, FN: {cm_initial[1,0]}")


## Classification Report
Final per-class performance metrics demonstrating comprehensive model effectiveness across both classes.

In [ ]:
print("\n" + "="*70)
print("DETAILED CLASSIFICATION REPORTS")
print("="*70)
print(classification_report(y_test, y_pred_initial, target_names=['Not Bestseller', 'Bestseller']))
print("="*70)

## ROC Curve
Final ROC curve and AUC visualisation assessing discrimination ability on test set.

In [ ]:
# Calculate ROC curve
fpr_initial, tpr_initial, roc_thresholds_initial = roc_curve(y_test, y_proba_initial)

# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr_initial, tpr_initial, label=f'ROC Curve (AUC = {roc_auc_initial:.3f})', 
         color='purple', linewidth=3)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Initial Model: ROC Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Precision-Recall Curve
Final precision-recall curve demonstrating performance characteristics particularly relevant for imbalanced classification.

In [ ]:
# Calculate Precision-Recall curve
precision_curve_initial, recall_curve_initial, pr_thresholds = precision_recall_curve(y_test, y_proba_initial)
# Calculate average precision
avg_precision_initial = average_precision_score(y_test, y_proba_initial)

# Plot Precision-Recall Curve
plt.figure(figsize=(8, 6))
plt.plot(recall_curve_initial, precision_curve_initial, label=f'Precision-Recall Curve ({avg_precision_initial:.3f})', 
         color='darkorange', linewidth=3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Initial Model: Precision-Recall Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower left")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAverage Precision Score: {avg_precision_initial:.4f}")


## Training Loss Curve
Final loss curve visualisation demonstrating convergence of optimised model.

In [ ]:
# Plot training loss curve
plt.figure(figsize=(10, 6))
plt.plot(initial_model.loss_history, linewidth=2, color='blue')
plt.title('Initial Model - Training Loss Curve', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Model Improvements and Retraining
Optimisation phase systematically improving model performance through hyperparameter tuning, class imbalance mitigation, and iterative retraining with refined parameters based on cross-validation performance.

## Hyperparameter Tuning
Hyperparameter search space definition specifies parameter ranges to explore: learning rates controlling optimisation step sizes, L2 regularisation coefficients controlling overfitting penalty strength, and iteration counts controlling training epochs. Systematic grid search exhaustively evaluates all parameter combinations.

In [ ]:
def cross_val_score_scratch(model_class, X, y, params, cv=5):
    """
    Cross-validation for custom model. Returns list of scores for F1 metric.
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]

        # Train model
        model = model_class(**params, verbose=False)
        model.fit(X_train_cv, y_train_cv)

        # Evaluate
        y_pred_cv = model.predict(X_val_cv, threshold=0.5)
        score = f1_score(y_val_cv, y_pred_cv, zero_division=0)
        scores.append(score)

        if fold == 0:  # Print first fold details
            print(f"  Fold 1 - F1 Score: {score:.4f}")

    return scores

# Define parameter grid
param_grid = {
    'lr': [0.001, 0.01, 0.05],
    'lambda_reg': [0.01, 0.1, 1.0],
    'epochs': [500, 1000, 2000]
}

print("\nParameter Grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

total_combinations = len(param_grid['lr']) * len(param_grid['lambda_reg']) * len(param_grid['epochs'])
print(f"\nTotal parameter combinations to test: {total_combinations}")

## Grid Search Execution
Grid search with cross-validation systematically evaluates all hyperparameter combinations across stratified folds computing F1-score as primary evaluation metric. Best parameters maximising cross-validation F1-score are selected for final model training.

In [ ]:
# Execute Grid Search
params_product = list(product(param_grid['lr'], param_grid['lambda_reg'], param_grid['epochs']))
best_score = 0
best_params = None
cv_results = []

print("\nRunning grid search with 5-fold cross-validation...")
print("-" * 70)

for i, (lr, lambda_reg, epochs) in enumerate(params_product):
    params = {
        'lr': lr,
        'lambda_reg': lambda_reg,
        'epochs': epochs
    }

    # Perform cross-validation
    scores = cross_val_score_scratch(LogisticRegressionScratch, X_train_scaled, y_train, params)
    mean_score = np.mean(scores)
    std_score = np.std(scores)

    cv_results.append({
        'params': params,
        'mean_f1': mean_score,
        'std_f1': std_score,
        'all_scores': scores
    })

    print(f"Config {i+1:2d}/{total_combinations}: lr={lr:.3f}, λ={lambda_reg:.2f}, epochs={epochs:4d} | "
          f"F1: {mean_score:.4f} (±{std_score:.4f})")

    # Track best parameters
    if mean_score > best_score:
        best_score = mean_score
        best_params = params

print("\n" + "="*50)
print("BEST HYPERPARAMETERS FOUND:")
print("="*50)
print(f"Learning Rate: {best_params['lr']}")
print(f"Regularization (λ): {best_params['lambda_reg']}")
print(f"Epochs: {best_params['epochs']}")
print(f"Best mean F1-score (CV): {best_score:.4f}")

## Bias-Variance Analysis
Learning curve analysis examining training and validation performance across increasing training set sizes diagnoses model complexity issues. Converging training and validation curves at high performance indicate good model fit. Diverging curves indicate overfitting or underfitting requiring corrective action.

In [ ]:
# Train model with best parameters to analyze learning curves
print("Training model with best parameters for learning curve analysis...")
final_model = LogisticRegressionScratch(**best_params, verbose=True)
final_model.fit(X_train_scaled, y_train)

# Analyze convergence
final_loss = final_model.loss_history[-1]
initial_loss = final_model.loss_history[0]
loss_reduction = (initial_loss - final_loss) / initial_loss * 100

print(f"\nLoss Analysis:")
print(f"  Initial Loss: {initial_loss:.4f}")
print(f"  Final Loss: {final_loss:.4f}")
print(f"  Loss Reduction: {loss_reduction:.1f}%")

# Check for convergence in last 100 epochs
if len(final_model.loss_history) >= 100:
    last_100_losses = final_model.loss_history[-100:]
    loss_std = np.std(last_100_losses)
    if loss_std < 0.001:
        print(" Status: Model converged (loss stabilized)")
    else:
        print(" Status: Model may benefit from more epochs")
else:
    print(" Status: Model trained for fewer than 100 epochs")

## Class Imbalance Mitigation
Class imbalance analysis quantifying training set class distribution informs mitigation strategy selection. Severe imbalance (e.g., 80-20 split) requires resampling through oversampling minority class or undersampling majority class to improve model fairness and minority class performance.

In [ ]:
print("Original training set distribution:")
original_counts = np.bincount(y_train)
print(f"  Class 0: {original_counts[0]} ({original_counts[0]/len(y_train)*100:.1f}%)")
print(f"  Class 1: {original_counts[1]} ({original_counts[1]/len(y_train)*100:.1f}%)")

# Apply SMOTE to training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

print(f"\nResampled training set:")
print(f"  Original size: {X_train_scaled.shape[0]} samples")
print(f"  Resampled size: {X_train_resampled.shape[0]} samples")

resampled_counts = np.bincount(y_train_resampled)
print(f"\nResampled class distribution:")
print(f"  Class 0: {resampled_counts[0]} ({resampled_counts[0]/len(y_train_resampled)*100:.1f}%)")
print(f"  Class 1: {resampled_counts[1]} ({resampled_counts[1]/len(y_train_resampled)*100:.1f}%)")

## Final Model Training (with Resampled Data)
Final model training using optimal hyperparameters and balanced training data through resampling. This model incorporates all improvements from previous phases.

In [ ]:
final_model_resampled = LogisticRegressionScratch(**best_params, verbose=True)
final_model_resampled.fit(X_train_resampled, y_train_resampled)

# Final Evaluation and Interpretation
Comprehensive final evaluation of optimised model examining predictions, performance metrics, and feature importance across all evaluation perspectives.

## Final Model Predictions and Threshold Optimization
Probability prediction and threshold optimisation selecting classification threshold maximising F1-score rather than assuming default 0.5 threshold. Threshold optimisation accommodates specific business requirements and class imbalance characteristics.

In [ ]:
# Get predicted probabilities for the positive class
y_pred_proba_final = final_model_resampled.predict_proba(X_test_scaled)

# Find optimal threshold based on F1 score
f1_scores_by_threshold = []
thresholds_to_test = np.arange(0.1, 0.9, 0.05)
# Created to metrics for each threshold 
results = []

best_threshold = 0.5
best_f1_threshold = 0.0

for threshold in thresholds_to_test:
    y_pred_thresh = (y_pred_proba_final >= threshold).astype(int)
    f1_thresh = f1_score(y_test, y_pred_thresh, zero_division=0)
    precision_thresh = precision_score(y_test, y_pred_thresh, zero_division=0)
    recall_thresh = recall_score(y_test, y_pred_thresh, zero_division=0)

    f1_scores_by_threshold.append(f1_thresh)
    
    # Will be used for Visualisation of Threshold Optimizations Visualization
    results.append({
        'threshold': threshold,
        'f1_score': f1_thresh,
        'precision': precision_thresh,
        'recall': recall_thresh
    })
    
    if f1_thresh > best_f1_threshold:
        best_f1_threshold = f1_thresh
        best_threshold = threshold

print(f"\n Optimal threshold for F1-Score: {best_threshold:.2f}")
print(f"   Best F1-Score achieved: {best_f1_threshold:.4f}")

# Make predictions using optimal threshold
y_pred_final = (y_pred_proba_final >= best_threshold).astype(int)


In [ ]:
print(f"\nFinal model predictions using optimal threshold:")
print(f"  Predictions shape: {y_pred_final.shape}")
print(f"  Probability range: [{y_pred_proba_final.min():.3f}, {y_pred_proba_final.max():.3f}]")
print(f"  Predicted classes: {np.unique(y_pred_final)}")

# Count predicted classes
final_pred_counts = np.bincount(y_pred_final)
print(f"\nFinal prediction distribution:")
print(f"  Predicted Non-Bestsellers: {final_pred_counts[0]} ({final_pred_counts[0]/len(y_pred_final)*100:.1f}%)")
print(f"  Predicted Bestsellers: {final_pred_counts[1]} ({final_pred_counts[1]/len(y_pred_final)*100:.1f}%)")

## Final Metrics Calculation
Final performance metrics computation across test set using optimised threshold quantifying model generalisation capability.

In [ ]:
accuracy_final = accuracy_score(y_test, y_pred_final) 
precision_final = precision_score(y_test, y_pred_final, zero_division=0) 
recall_final = recall_score(y_test, y_pred_final, zero_division=0) 
f1_final = f1_score(y_test, y_pred_final, zero_division=0) 
roc_auc_final = roc_auc_score(y_test, y_pred_proba_final)

print("MODEL PERFORMANCE")
print(f"  Accuracy:  {accuracy_final:.4f}")
print(f"  Precision: {precision_final:.4f}")
print(f"  Recall:    {recall_final:.4f}")
print(f"  F1-Score:  {f1_final:.4f}")
print(f"  ROC AUC:   {roc_auc_final:.4f}")

# Confusion Matrix
cm_final = confusion_matrix(y_test, y_pred_final)
print("\nConfusion Matrix:")
print(cm_final)
print("\nConfusion Matrix Breakdown:")
tn, fp, fn, tp = cm_final.ravel()
print(f"  True Negatives (TN):  {tn}")
print(f"  False Positives (FP): {fp}")
print(f"  False Negatives (FN): {fn}")
print(f"  True Positives (TP):  {tp}")

## Confusion Matrix Visualization
Final confusion matrix presentation showing classification performance on test set with optimised threshold.

In [ ]:
# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_final, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Bestseller', 'Bestseller'],
            yticklabels=['Non-Bestseller', 'Bestseller'])
plt.title('Final Model - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

## Classification Report
Final per-class performance metrics demonstrating comprehensive model effectiveness across both classes.

In [ ]:
# Classification Report
print("\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_test, y_pred_final, 
                          target_names=['Non-Bestseller', 'Bestseller']))

## ROC Curve
Final ROC curve and AUC visualisation assessing discrimination ability on test set.

In [ ]:
# Calculate ROC curve
fpr_final, tpr_final, roc_thresholds_final = roc_curve(y_test, y_pred_proba_final)

# Plot ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr_final, tpr_final, label=f'ROC Curve (AUC = {roc_auc_final:.3f})', 
         color='purple', linewidth=3)
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Final Model: ROC Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Precision-Recall Curve
Final precision-recall curve demonstrating performance characteristics particularly relevant for imbalanced classification.

In [ ]:
# Calculate Precision-Recall curve
precision_curve_final, recall_curve_final, pr_thresholds = precision_recall_curve(y_test, y_pred_proba_final)
# Calculate average precision
avg_precision_final = average_precision_score(y_test, y_pred_proba_final)

# Plot Precision-Recall Curve
plt.figure(figsize=(8, 6))
plt.plot(recall_curve_final, precision_curve_final, label=f'Precision-Recall Curve ({avg_precision_final:.3f})', 
         color='darkorange', linewidth=3)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Final Model: Precision-Recall Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower left")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


print(f"\nAverage Precision Score: {avg_precision_final:.4f}")


## Training Loss Curve
Final loss curve visualisation demonstrating convergence of optimised model.

In [ ]:
# Plot learning curves
plt.figure(figsize=(10, 6))
plt.plot(final_model.loss_history, linewidth=2, color='green')
plt.title('Final Model - Training Loss Curve', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Threshold Optimization Visualization
Threshold optimisation results visualisation showing F1-score variation across probability thresholds, demonstrating threshold selection rationale.

In [ ]:
results_df = pd.DataFrame(results)

# Visualize the threshold optimization
plt.figure(figsize=(10, 6))
plt.plot(results_df['threshold'], results_df['f1_score'], marker='o', label='F1-Score')
plt.plot(results_df['threshold'], results_df['precision'], marker='s', label='Precision')
plt.plot(results_df['threshold'], results_df['recall'], marker='^', label='Recall')
plt.xlabel('Classification Threshold')
plt.ylabel('Score')
plt.title('Metrics vs Classification Threshold')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Initial and Final Model Comparison and Evaluation
Comparative analysis between initial baseline and optimised final model quantifying improvements across all metrics.

## Model Comparison Table
Summary table juxtaposing initial and final model metrics enabling rapid visual comparison of improvements.

In [ ]:
# Create comparison table
metrics_comparison = pd.DataFrame({
    'Initial Model': [accuracy_initial, precision_initial, recall_initial, f1_initial, roc_auc_initial],
    'Final Model': [accuracy_final, precision_final, recall_final, f1_final, roc_auc_final],
    'Improvement': [accuracy_final-accuracy_initial, precision_final-precision_initial, 
                   recall_final-recall_initial, f1_final-f1_initial, roc_auc_final-roc_auc_initial]
}, index=['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC'])

print("\nDetailed Model Comparison:")
print(metrics_comparison.round(4))

# Determine best improvements
improvements = metrics_comparison['Improvement']
best_improvement = improvements.idxmax()
worst_improvement = improvements.idxmin()

print(f"\n Best Improvement: {best_improvement} (+{improvements[best_improvement]:.4f})")
print(f" Worst Change: {worst_improvement} ({improvements[worst_improvement]:+.4f})")

## Comparitive Charts
Bar chart visualisation comparing model metrics side-by-side across accuracy, precision, recall, F1-score, and ROC AUC.

In [ ]:
# Metrics
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC AUC']
original = [accuracy_initial, precision_initial, recall_initial, f1_initial, roc_auc_initial]
final = [accuracy_final, precision_final, recall_final, f1_final, roc_auc_final]

# Side-by-side bar chart
x = np.arange(len(metrics))
width = 0.35

plt.figure(figsize=(12,6))
plt.bar(x - width/2, original, width, label='Original', color='lightblue')
plt.bar(x + width/2, final, width, label='Final', color='orange')
plt.xticks(x, metrics)
plt.ylabel('Score')
plt.ylim(0, 1.05)
plt.title('Model Performance: Original vs Final')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Delta bar chart
deltas = [f - o for o, f in zip(original, final)]
colors = ['green' if d > 0 else 'red' for d in deltas]

plt.figure(figsize=(12,6))
plt.bar(metrics, deltas, color=colors)
plt.axhline(0, color='black', linestyle='--')
plt.ylabel('Change (Δ)')
plt.title('Change in Model Performance After Retraining')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Curve Comparisons
Superimposed curve comparison showing initial versus final model performance across ROC and precision-recall curves.

### ROC Curve
ROC curve comparison demonstrating discriminative ability improvement from initial to final model.

In [ ]:
# Compare Initial vs Final ROC Curves
plt.figure(figsize=(10, 6))
plt.plot(fpr_initial, tpr_initial, color='blue', lw=2, 
         label=f'Initial ROC (AUC = {roc_auc_initial:.3f})')
plt.plot(fpr_final, tpr_final, color='orange', lw=2, 
         label=f'Final ROC (AUC = {roc_auc_final:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve: Initial vs Final Comparison')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nROC-AUC Initial: {roc_auc_initial:.4f}")
print(f"ROC-AUC Final:  {roc_auc_final:.4f}")
print(f"AUC Gap:       {abs(roc_auc_initial - roc_auc_final):.4f}")


### Precision-Recall Curve
Precision-recall curve comparison showing performance improvement particularly relevant for imbalanced classification.

In [ ]:
# Compare Initial vs Final Precision-Recall Curves
plt.figure(figsize=(10, 6))
plt.plot(recall_curve_initial, precision_curve_initial, color='blue', lw=2, 
         label=f'Initial PR (AP = {avg_precision_initial:.3f})')
plt.plot(recall_curve_final, precision_curve_final, color='orange', lw=2, 
         label=f'Final PR (AP = {avg_precision_final:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve: Initial vs Final Comparison')
plt.legend(loc="lower left")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nAverage Precision Initial: {avg_precision_initial:.4f}")
print(f"Average Precision Final:  {avg_precision_final:.4f}")
print(f"AP Gap:                  {abs(avg_precision_initial - avg_precision_final):.4f}")


## Feature Interpretation
Feature coefficient analysis ranking features by magnitude and sign interpretation. Positive coefficients increase predicted probability of target=True whilst negative coefficients decrease it. Coefficient magnitudes reflect feature importance in model decisions.

In [ ]:
coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': final_model_resampled.get_coefficients()['weights'],
    'Abs_Coefficient': np.abs(final_model_resampled.get_coefficients()['weights'])
}).sort_values('Abs_Coefficient', ascending=False)

print("\n=== Feature Importance (by coefficient magnitude) ===")
print(coefficients)

# Visualize top features
plt.figure(figsize=(10, 8))
top_n = min(15, len(coefficients))
top_features = coefficients.head(top_n)

# Color: green if positive, red if negative
colors = ['green' if x > 0 else 'red' for x in top_features['Coefficient']]

plt.barh(range(top_n), top_features['Coefficient'], color=colors)
plt.yticks(range(top_n), top_features['Feature'])
plt.xlabel('Coefficient Value')
plt.title('Top Feature Coefficients (Green=Positive, Red=Negative)')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

# References

Dictionary.com. (2023). Dictionary.com | Meanings & Definitions of English Words. [online] Available at: https://www.dictionary.com/browse/bestseller [Accessed 30 Oct. 2025].

‌